In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Set up the WebDriver with headless option
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Uncomment for headless mode
options.add_argument('--no-sandbox')  # Recommended option for headless mode
options.add_argument('--disable-dev-shm-usage')  # Disable shared memory usage for better stability

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Start with the first page
base_url = 'https://www.ajio.com/s/perfumes-4405-57701?query=:relevance&classifier=intent'
driver.get(base_url)

# Initialize an empty list to store product data
product_data = []
max_products = 200  # Set limit to stop after collecting a certain number of products

def extract_product_details():
    products = driver.find_elements(By.CSS_SELECTOR, 'div.item.rilrtl-products-list__item')

    for product in products:
        if len(product_data) >= max_products:  # Stop if we have enough data
            return False

        product_info = {}

        try:
            # Extract product link (using correct CSS selector)
            link_elements = product.find_elements(By.CSS_SELECTOR, 'a')
            product_link = link_elements[0].get_attribute('href') if link_elements else 'No link'
            product_info['link'] = product_link
        except Exception:
            product_info['link'] = 'No link'

        # Now go to the individual product page to extract details
        if product_info['link'] != 'No link':
            try:
                # Open product link in a new tab
                driver.execute_script(f"window.open('{product_info['link']}', '_blank');")
                driver.switch_to.window(driver.window_handles[1])

                # Wait for the product page to load
                time.sleep(2)

                # Extract title
                try:
                    title_elements = driver.find_elements(By.CSS_SELECTOR, 'h2.brand-name')
                    product_info['title'] = title_elements[0].text if title_elements else 'No title'
                except Exception:
                    product_info['title'] = 'No title'

                # Extract price
                try:
                    price_elements = driver.find_elements(By.CSS_SELECTOR, 'div.prod-sp')
                    product_info['price'] = price_elements[0].text if price_elements else 'No price'
                except Exception:
                    product_info['price'] = 'No price'

                # Extract ratings
                try:
                    rating_elements = driver.find_elements(By.CSS_SELECTOR, 'span._3c5q0')
                    product_info['rating'] = rating_elements[0].text if rating_elements else 'No rating'
                except Exception:
                    product_info['rating'] = 'No rating'

                # Extract number of reviews
                try:
                    reviews_elements = driver.find_elements(By.CSS_SELECTOR, 'span._38RNg')
                    product_info['reviews'] = reviews_elements[0].text if reviews_elements else 'No reviews'
                except Exception:
                    product_info['reviews'] = 'No reviews'

                print(f"Scraped details from product page: {product_info}")

                # Close the product tab and switch back to the search results tab
                driver.close()
                driver.switch_to.window(driver.window_handles[0])

                # Wait for the page to return to the main search
                time.sleep(2)

            except Exception as e:
                print(f"Error while scraping product page: {e}")
                product_info['title'] = 'No title'
                product_info['price'] = 'No price'
                product_info['rating'] = 'No rating'
                product_info['reviews'] = 'No reviews'

        # Append the product info to the list
        product_data.append(product_info)

        # Print product information to show the scraping progress
        print(f"Scraped product {len(product_data)}: {product_info}")

    return True  # Continue scraping


# Scroll the page and extract products until we reach the max product limit
scroll_pause_time = 2  # Adjust if needed for smoother scrolling
while len(product_data) < max_products:
    extract_success = extract_product_details()

    if not extract_success:
        break

    # Scroll down to load more products
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the page to load new products
    time.sleep(scroll_pause_time)

# Close the driver after scraping
driver.quit()

# Convert the product data into a DataFrame and save it to an Excel file
df = pd.DataFrame(product_data)
df.to_excel('ajio_perfume.xlsx', index=False)  # Use .to_excel() for Excel format
print('Data saved to ajio_perfume.xlsx.')

Scraped details from product page: {'link': 'https://www.ajio.com/superdry-sport-vegan-revive-body-spray/p/410270323_yellow', 'title': 'SUPERDRY', 'price': 'MRP₹499', 'rating': 'No rating', 'reviews': 'No reviews'}
Scraped product 1: {'link': 'https://www.ajio.com/superdry-sport-vegan-revive-body-spray/p/410270323_yellow', 'title': 'SUPERDRY', 'price': 'MRP₹499', 'rating': 'No rating', 'reviews': 'No reviews'}
Scraped details from product page: {'link': 'https://www.ajio.com/superdry-vegan-body-spray--vetiver-bergamot-woods/p/410270333_hawaiip', 'title': 'SUPERDRY', 'price': 'MRP₹499', 'rating': '3.9', 'reviews': '92 Ratings'}
Scraped product 2: {'link': 'https://www.ajio.com/superdry-vegan-body-spray--vetiver-bergamot-woods/p/410270333_hawaiip', 'title': 'SUPERDRY', 'price': 'MRP₹499', 'rating': '3.9', 'reviews': '92 Ratings'}
Scraped details from product page: {'link': 'https://www.ajio.com/bella-vita-organic-luxury-perfumes-gift-set-for-women/p/4932363390_multi', 'title': 'BELLA VIT

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=128.0.6613.120)
Stacktrace:
	GetHandleVerifier [0x00D98213+26163]
	(No symbol) [0x00D29CC4]
	(No symbol) [0x00C224C3]
	(No symbol) [0x00C0D118]
	(No symbol) [0x00C0D039]
	(No symbol) [0x00C24960]
	(No symbol) [0x00CA3FB9]
	(No symbol) [0x00C8AA56]
	(No symbol) [0x00C5BE89]
	(No symbol) [0x00C5C8CD]
	GetHandleVerifier [0x0106D313+2996019]
	GetHandleVerifier [0x010C1B89+3342249]
	GetHandleVerifier [0x00E27AEF+614159]
	GetHandleVerifier [0x00E2F17C+644508]
	(No symbol) [0x00D327FD]
	(No symbol) [0x00D2F6F8]
	(No symbol) [0x00D2F895]
	(No symbol) [0x00D21C16]
	BaseThreadInitThunk [0x76167BA9+25]
	RtlInitializeExceptionChain [0x777BC11B+107]
	RtlClearBits [0x777BC09F+191]


In [2]:
# Convert the product data into a DataFrame and save it to an Excel file
df = pd.DataFrame(product_data)
df.to_excel('ajio_perfume.xlsx', index=False)  # Use .to_excel() for Excel format
print('Data saved to ajio_perfume.xlsx.')

Data saved to ajio_perfume.xlsx.
